In [2]:
from bs4 import BeautifulSoup
import requests as requests

**I have to insert a try except because some posts do not have a title**

In [24]:
titles = []

for i in range(1, 100):
    
    try:
        url_posts = 'https://stacker.news/items/%d' % (i)
        response = requests.get(url_posts)
        soup = BeautifulSoup(response.text, 'html.parser')
    
        title = soup.find_all('a', class_='item_title__FH7AS text-reset me-2')
        
        for e in title:
            titles.append(e.get_text())
        
    except:
        continue
        

In [27]:
titles

['El Salvador Makes Bitcoin Legal Tender',
 'stacker news soft launch AMA',
 'The Lightning Conference 2019: Alex Bosworth',
 'Pieter Wuille on the Taproot Genesis Story ',
 'Next Bitcoin Soft-Fork ANYPREVOUT',
 'Fedcoin is coming',
 "Why Altcoins aren't copying Taproot. Bitcoin Tech Talk #244",
 "Bitrefill's Work in El Salvador",
 '3 Ways El Salvador Has Changed Bitcoin Forever',
 'Russell Okung asks the Nigerian government to adopt a Bitcoin standard',
 'Bitcoin Optech Newsletter #152',
 'The Colonial Pipeline hack - what actually happened',
 'China Miner Shutdown Actually Being Enforced',
 'Stranded Energy and Home Mining',
 'Submarine swaps and Loops (Lightning Network) with Alex Bosworth',
 'Stacks Blockchain Explanation ',
 'Lightning Ubiquity and the United Nations of Bitcoin',
 'Stephan Livera live tweets "Bitcoin Mining Council" meeting',
 'Lightning Dev Kit',
 'Texas Clarifies that State-chartered Banks can Custody Bitcoin',
 'Taproot Derivation Support for Descriptors',
 'Re